In [ ]:
import myCrawler as crawler

url = 'https://explorer-web.api.btc.com/v1/eth/miner/stats/list?time_range=4&page=1&size=20'


In [ ]:
r = crawler.myRequests(url).json()

In [ ]:
def lengthOfLongestSubstring(s):
    """
    :type s: str
    :rtype: int
    """
    max_length = 0             
    for i in range(len(s)): 
        tmp = set()
        for j in range(i, len(s)):
            tmp.add(s[j])
            if len(tmp) > max_length:
                max_length = len(tmp)
            if len(tmp) != j-i+1:
                break                
            
    return max_length

In [ ]:
def rewards(init_fund, invest_limit, y_return, period):
    if period == 1:
        return init_fund*(1+y_return)+invest_limit
    else:
        return rewards(init_fund, invest_limit, y_return, period-1)*(1+y_return)+invest_limit

In [ ]:
rewards(20, 10, 0.25, 5)

In [ ]:
import pandas as pd

df = pd.read_csv('/Users/li/Downloads/sp500.csv')
df.head()

In [ ]:
df = df.sort_values(by = 'Date',ascending=False)
df.head()

In [ ]:
len(df)

In [ ]:
df.tail()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(18,8))
plt.plot(pd.to_datetime(df['Date']), df['Close'])

In [ ]:
import numpy as np
import pandas as pd

class DataLoader():
    """A class for loading and transforming data for the lstm model"""

    def __init__(self, filename, split, cols):
        dataframe = pd.read_csv(filename)
        i_split = int(len(dataframe) * split)
        self.data_train = dataframe.get(cols).values[:i_split]
        self.data_test  = dataframe.get(cols).values[i_split:]
        self.len_train  = len(self.data_train)
        self.len_test   = len(self.data_test)
        self.len_train_windows = None
        
    def y_multiclass(self, x): 
        '''相对于窗口第一天涨幅进行分类'''
        pp = x*100
        if pp < -10:
            return 0
        elif pp < -5:
            return 1
        elif pp < 0:
            return 2
        elif pp < 5:
            return 3
        elif pp < 10:
            return 4
        else:
            return 5 

    def get_test_data(self, seq_len, normalise):
        '''
        Create x, y test data windows
        Warning: batch method, not generative, make sure you have enough memory to
        load data, otherwise reduce size of the training split.
        '''
        data_windows = []
        for i in range(self.len_test - seq_len):
            data_windows.append(self.data_test[i:i+seq_len])

        data_windows = np.array(data_windows).astype(float)
        data_windows = self.normalise_windows(data_windows, single_window=False) if normalise else data_windows
        
        x = data_windows[:, :-1]
        y = data_windows[:, -1, [0]]
        y = [[self.y_multiclass(ele[0])] for ele in y]
        return x,y

    def get_train_data(self, seq_len, normalise):
        '''
        Create x, y train data windows
        Warning: batch method, not generative, make sure you have enough memory to
        load data, otherwise use generate_training_window() method.
        '''
        data_x = []
        data_y = []
        for i in range(self.len_train - seq_len):
            x, y = self._next_window(i, seq_len, normalise)
            data_x.append(x)
            data_y.append(y)
        return np.array(data_x), np.array(data_y)

    def generate_train_batch(self, seq_len, batch_size, normalise):
        '''Yield a generator of training data from filename on given list of cols split for train/test'''
        i = 0
        while i < (self.len_train - seq_len):
            x_batch = []
            y_batch = []
            for b in range(batch_size):
                if i >= (self.len_train - seq_len):
                    # stop-condition for a smaller final batch if data doesn't divide evenly
                    yield np.array(x_batch), np.array(y_batch)
                    i = 0
                x, y = self._next_window(i, seq_len, normalise)
                x_batch.append(x)
                y_batch.append(y)
                i += 1
            yield np.array(x_batch), np.array(y_batch)

    def _next_window(self, i, seq_len, normalise):
        '''Generates the next data window from the given index location i'''
        window = self.data_train[i:i+seq_len]
        window = self.normalise_windows(window, single_window=True)[0] if normalise else window
        x = window[:-1]
        y = window[-1, [0]]
        
        y = [self.y_multiclass(ele) for ele in y]
        return x, y

    def normalise_windows(self, window_data, single_window=False):
        '''Normalise window with a base value of zero'''
        normalised_data = []
        window_data = [window_data] if single_window else window_data
        for window in window_data:
            
            normalised_window = []
            for col_i in range(window.shape[1]):
                normalised_col = [((float(p) / float(window[0, col_i])) - 1) for p in window[:, col_i]]
                
                normalised_window.append(normalised_col)
            
            normalised_window = np.array(normalised_window).T # reshape and transpose array back into original multidimensional format
            normalised_data.append(normalised_window)
        return np.array(normalised_data)

In [ ]:
data = DataLoader("/Users/li/LSTM-Neural-Network-for-Time-Series-Prediction/data/sp500.csv",0.85,['Close','Volume'])